<a href="https://colab.research.google.com/github/gromdimon/code/blob/main/Bioprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioactivity project**


# Installing libraries

In [ ]:
! pip install chembl_webresource_client

     |████████████████████████████████| 61kB 4.5MB/s 
  Created wheel for chembl-webresource-client: filename=chembl_webresource_client-0.10.4-cp37-none-any.whl size=55663 sha256=f5f032e0364fb08d3ef57999e77919a6b84013f664190eaf7a7d4ce12b3d4fc1
  Stored in directory: /root/.cache/pip/wheels/98/4c/16/bdb2ef9c937173e56b04d71fc1a6f98f0ff24992ae2e35daf3
Successfully built chembl-webresource-client


# Importing libraries

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Searching target protein

In [ ]:
target = new_client.target
target_query = target.search('Beta amyloid A4 protein')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Rattus norvegicus,Amyloid beta A4 protein,33.0,False,CHEMBL3638365,"[{'accession': 'P08592', 'component_descriptio...",SINGLE PROTEIN,10116.0
1,[],Mus musculus,Amyloid-beta A4 protein,33.0,False,CHEMBL4523942,"[{'accession': 'P12023', 'component_descriptio...",SINGLE PROTEIN,10090.0
2,"[{'xref_id': 'P05067', 'xref_name': None, 'xre...",Homo sapiens,Beta amyloid A4 protein,29.0,False,CHEMBL2487,"[{'accession': 'P05067', 'component_descriptio...",SINGLE PROTEIN,9606.0
3,"[{'xref_id': 'P37840', 'xref_name': None, 'xre...",Homo sapiens,Alpha-synuclein,29.0,False,CHEMBL6152,"[{'accession': 'P37840', 'component_descriptio...",SINGLE PROTEIN,9606.0
4,[],Homo sapiens,Amyloid beta-binding alcohol dehydrogenase,19.0,False,CHEMBL4295598,"[{'accession': 'Q2L8D9', 'component_descriptio...",SINGLE PROTEIN,9606.0
...,...,...,...,...,...,...,...,...,...
4749,[],Homo sapiens,PIK3CG/PIK3R1,0.0,False,CHEMBL4296106,"[{'accession': 'P27986', 'component_descriptio...",PROTEIN COMPLEX,9606.0
4750,[],Homo sapiens,NTRK1/NTRK2,0.0,False,CHEMBL4523622,"[{'accession': 'P04629', 'component_descriptio...",PROTEIN FAMILY,9606.0
4751,[],Homo sapiens,Secretory phospholipase A2,0.0,False,CHEMBL4524005,"[{'accession': 'Q5R387', 'component_descriptio...",PROTEIN FAMILY,9606.0
4752,[],Rattus norvegicus,P2X receptor,0.0,False,CHEMBL4524013,"[{'accession': 'P51579', 'component_descriptio...",PROTEIN FAMILY,10116.0


In [ ]:
# This is alternative way to sort dataset
# tg = target_query.filter(organism='Homo sapiens')
# tgs = pd.DataFrame.from_dict(tg)
# tgs

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P05067', 'xref_name': None, 'xre...",Homo sapiens,Beta amyloid A4 protein,29.0,False,CHEMBL2487,"[{'accession': 'P05067', 'component_descriptio...",SINGLE PROTEIN,9606
1,"[{'xref_id': 'P37840', 'xref_name': None, 'xre...",Homo sapiens,Alpha-synuclein,29.0,False,CHEMBL6152,"[{'accession': 'P37840', 'component_descriptio...",SINGLE PROTEIN,9606
2,[],Homo sapiens,Amyloid beta-binding alcohol dehydrogenase,19.0,False,CHEMBL4295598,"[{'accession': 'Q2L8D9', 'component_descriptio...",SINGLE PROTEIN,9606
3,[],Homo sapiens,NEDD8-activating enzyme E1 regulatory subunit,18.0,False,CHEMBL2016431,"[{'accession': 'Q13564', 'component_descriptio...",SINGLE PROTEIN,9606
4,[],Homo sapiens,Protein S100-A4,18.0,False,CHEMBL2362976,"[{'accession': 'P26447', 'component_descriptio...",SINGLE PROTEIN,9606
...,...,...,...,...,...,...,...,...,...
2876,[],Homo sapiens,Prostaglandin E synthase/G/H synthase 2,0.0,False,CHEMBL3885623,"[{'accession': 'P35354', 'component_descriptio...",PROTEIN FAMILY,9606
2877,[],Homo sapiens,Mannan-binding lectin serine protease 1,0.0,False,CHEMBL4295768,"[{'accession': 'P48740', 'component_descriptio...",SINGLE PROTEIN,9606
2878,[],Homo sapiens,PIK3CG/PIK3R1,0.0,False,CHEMBL4296106,"[{'accession': 'P27986', 'component_descriptio...",PROTEIN COMPLEX,9606
2879,[],Homo sapiens,NTRK1/NTRK2,0.0,False,CHEMBL4523622,"[{'accession': 'P04629', 'component_descriptio...",PROTEIN FAMILY,9606


# We chose one protein as selected_target

In [ ]:
selected_target = targets.target_chembl_id[2]
selected_target

'CHEMBL2487'

# Now we will see bioactivity of CHEMBL2487

In [ ]:
activity = new_client.activity
chemble_activity = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
act = pd.DataFrame.from_dict(chemble_activity)
act

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,357577,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CCC(C1)C(C)(C)C2NS(=O)(=O)c1ccc(F)cc1,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '17.02', 'le': '0.34', 'lle': '1.98', ...",CHEMBL311039,None,CHEMBL311039,5.30,False,http://www.openphacts.org/units/Nanomolar,132837,=,1,True,=,None,IC50,nM,None,5000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,5.0
1,None,357580,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1cccs1,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '18.60', 'le': '0.40', 'lle': '2.33', ...",CHEMBL450926,None,CHEMBL450926,5.57,False,http://www.openphacts.org/units/Nanomolar,132839,=,1,True,=,None,IC50,nM,None,2700.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,2.7
2,None,358965,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '18.45', 'le': '0.37', 'lle': '2.42', ...",CHEMBL310242,None,CHEMBL310242,5.75,False,http://www.openphacts.org/units/Nanomolar,132841,=,1,True,=,None,IC50,nM,None,1800.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,1.8
3,None,368887,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '15.12', 'le': '0.32', 'lle': '1.13', ...",CHEMBL74874,None,CHEMBL74874,4.96,False,http://www.openphacts.org/units/Nanomolar,132840,=,1,True,=,None,IC50,nM,None,11000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,11.0
4,None,375954,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '13.43', 'le': '0.33', 'lle': '1.06', ...",CHEMBL75183,None,CHEMBL75183,5.00,False,http://www.openphacts.org/units/Nanomolar,132838,=,1,True,=,None,IC50,nM,None,10000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,Not Active,20120045,[],CHEMBL4510291,APP40 inhibition assay,B,None,None,BAO_0000190,BAO_0000357,single protein format,CC(C)(C)OC(=O)N1CCCC1CNC1CCC(c2cc(F)ccc2F)(S(=...,None,None,CHEMBL4507288,None,NaN,"{'bei': '10.18', 'le': '0.21', 'lle': '-0.42',...",CHEMBL4558518,None,CHEMBL4558518,5.80,False,http://www.openphacts.org/units/Nanomolar,3359706,=,54,True,=,None,IC50,nM,None,1600.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,nM,UO_0000065,None,1600.0
1230,Active,20120642,"[{'comments': None, 'relation': None, 'result_...",CHEMBL4510293,APP42 inhibition assay,B,None,None,BAO_0000190,BAO_0000357,single protein format,C

In [ ]:
act.standard_type.unique()

array(['IC50'], dtype=object)

# Saving this bioactivity data as csv file

In [ ]:
act.to_csv('Beta_amyloid_bioactivity_data.csv', index=False)

# Making our data clearer

In [ ]:
act_normal = act[act.standard_value.notna()]
act_normal = act_normal[act_normal.canonical_smiles.notna()]
act_normal = act_normal.drop_duplicates(['canonical_smiles'])
act_normal

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,357577,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CCC(C1)C(C)(C)C2NS(=O)(=O)c1ccc(F)cc1,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '17.02', 'le': '0.34', 'lle': '1.98', ...",CHEMBL311039,None,CHEMBL311039,5.30,False,http://www.openphacts.org/units/Nanomolar,132837,=,1,True,=,None,IC50,nM,None,5000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,5.0
1,None,357580,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1cccs1,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '18.60', 'le': '0.40', 'lle': '2.33', ...",CHEMBL450926,None,CHEMBL450926,5.57,False,http://www.openphacts.org/units/Nanomolar,132839,=,1,True,=,None,IC50,nM,None,2700.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,2.7
2,None,358965,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '18.45', 'le': '0.37', 'lle': '2.42', ...",CHEMBL310242,None,CHEMBL310242,5.75,False,http://www.openphacts.org/units/Nanomolar,132841,=,1,True,=,None,IC50,nM,None,1800.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,1.8
3,None,368887,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '15.12', 'le': '0.32', 'lle': '1.13', ...",CHEMBL74874,None,CHEMBL74874,4.96,False,http://www.openphacts.org/units/Nanomolar,132840,=,1,True,=,None,IC50,nM,None,11000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,11.0
4,None,375954,[],CHEMBL678443,Inhibition of A-beta-42 production by inhibiti...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,None,None,CHEMBL1133739,J. Med. Chem.,2000.0,"{'bei': '13.43', 'le': '0.33', 'lle': '1.06', ...",CHEMBL75183,None,CHEMBL75183,5.00,False,http://www.openphacts.org/units/Nanomolar,132838,=,1,True,=,None,IC50,nM,None,10000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,None,19258715,[],CHEMBL4403499,Inhibition of beta amyloid in human SH-SY5Y cells,B,None,None,BAO_0000190,BAO_0000219,cell-based format,O=c1oc2c(O)c(O)cc3c(=O)oc4c(O)c(O)cc1c4c23,Outside typical range,Values for this activity type are unusually la...,CHEMBL4402535,Eur J Med Chem,2019.0,None,CHEMBL6246,ELLAGIC ACID,CHEMBL6246,None,False,http://www.openphacts.org/units/Nanomolar,3215485,=,1,True,=,None,IC50,nM,None,300000.0,CHEMBL2487,Homo sapiens,Beta amyloid A4 protein,9606,None,None,IC50,uM,UO_0000065,None,300.0
1226,Active,20120042,[],CHEMBL4510290,APP42 inhibition assay,B,None,None,BAO_0000190,BAO_0000357,single protein format,O=S(=O)(NC

# Making a new Dataframe

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
new_act = act_normal[selection]
new_act

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL311039,CC12CCC(C1)C(C)(C)C2NS(=O)(=O)c1ccc(F)cc1,5000.0
1,CHEMBL450926,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1cccs1,2700.0
2,CHEMBL310242,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,1800.0
3,CHEMBL74874,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,11000.0
4,CHEMBL75183,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,10000.0
...,...,...,...
1224,CHEMBL6246,O=c1oc2c(O)c(O)cc3c(=O)oc4c(O)c(O)cc1c4c23,300000.0
1226,CHEMBL1091513,O=S(=O)(NC1CCC(c2cc(F)ccc2F)(S(=O)(=O)c2ccc(Cl...,0.5
1229,CHEMBL4558518,CC(C)(C)OC(=O)N1CCCC1CNC1CCC(c2cc(F)ccc2F)(S(=...,1600.0
1230,CHEMBL3609637,COc1cc(-c2cn(C3CCc4c(F)cccc4N(CC(F)(F)F)C3=O)n...,31.0


# Saving this Dataframe to csv file

In [ ]:
new_act.to_csv('Beta_amyloid_bioactivity_clear', index=False)

# Making new column

In [ ]:
bioactivity_threshold = []
for i in new_act.standard_value:
  if float.i > 10000 